In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from datetime import datetime

In [2]:
driver = Chrome(executable_path='/Users/chrisjohnson/Downloads/chromedriver')
url = "https://the-luxury-collection.marriott.com/our-hotels/"
driver.get(url)
time.sleep(30)
# viewhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[3]/button')[0]
# viewhotels_button.click()
# northamhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[2]/pacsys-directory-listings/div/div[5]/a')[0]
# northamhotels_button.click()

In [3]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [4]:
stuff = soup.find_all('a', {'class', 'swell_vertical-xs'})
# stuff
hotel_list = []
for x in stuff:
    hotel = {}
    hotel['name'] = x.text.strip()
    hotel['url'] = x.attrs['href']
    hotel_list.append(hotel)
hotelsurl_df = pd.DataFrame(hotel_list)
hotelsurl_df

,name,url
0,"Sheraton Addis, a Luxury Collection Hotel, Add...",https://www.marriott.com/ADDLC
1,"North Island, a Luxury Collection Resort, Seyc...",https://www.marriott.com/SEZLC
2,"The Alexander, a Luxury Collection Hotel, Yerevan",https://www.marriott.com/EVNLC
3,"Meixi Lake Hotel, a Luxury Collection Hotel, C...",https://www.marriott.com/CSXLC
4,"The Castle Hotel, a Luxury Collection Hotel, D...",https://www.marriott.com/DLCLC
...,...,...
120,"The Equinox, a Luxury Collection Golf Resort &...",https://www.marriott.com/RUTLC
121,"Park Tower, a Luxury Collection Hotel, Buenos ...",https://www.marriott.com/BUEPL
122,"Palacio del Inka, a Luxury Collection Hotel, C...",https://www.marriott.com/CUZLC
123,"Hotel Paracas, a Luxury Collection Resort, Par...",https://www.marriott.com/PIOLC


In [5]:
%%time

hotel_list = []
counter = 0
for index, row in hotelsurl_df.iterrows():
    name = row['name']
    url = row['url']
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        stuff = soup.find('div', {'class' : 'l-container l-header-container'})
#             hotel_list = []
        hotel = {}
        try:
            hotel['brand'] = 'Luxury Collection'
        except:
            hotel['brand'] = 'Luxury Collection'
        try:
            hotel['name'] = name
        except:
            hotel['name'] = name
        try:
            hotel['url'] = url
        except:
            hotel['url'] = url
        try:
            hotel['street'] = stuff.find_all('span')[2].text
        except:
            hotel['street'] = 'N/A'
        try:
            hotel['locality'] = stuff.find_all('span')[4].text
        except:
            hotel['locality'] = 'N/A'        
        try:
            hotel['state'] = stuff.find_all('span')[6].text
        except:
            hotel['state'] = 'N/A'            
        try:
            hotel['postalcode'] = stuff.find_all('span')[7].text
        except:
            hotel['postalcode'] = 'N/A'            
        try:
            hotel['coordinate1'] = stuff.find_all('span')[25].text
        except:
            hotel['coordinate1'] = 'N/A'            
        try:
            hotel['coordinate2'] = stuff.find_all('span')[26].text
        except:
            hotel['coordinate2'] = 'N/A'
        try:
            hotel['coordinate3'] = stuff.find_all('span')[27].text
        except:
            hotel['coordinate3'] = 'N/A'            
        try:
            hotel['coordinate4'] = stuff.find_all('span')[28].text
        except:
            hotel['coordinate4'] = 'N/A'            
        
        desc = soup.find_all('p', {'class' : 'l-margin-top-none l-overflow-hidden t-collapsed-height'})
        try:
            hotel['description'] = str(desc).replace('<p class="l-margin-top-none l-overflow-hidden t-collapsed-height">','').replace('</p>', '')
        except:
            hotel['description'] = 'N/A'
        
        overallratings = stuff = soup.find('div', {'class' : 'l-overall-rating-container'})
        try:
            hotel['overall_rating'] = ratings.find_all('div')[0].text.strip()[:4]
        except:
            hotel['overall_rating'] = 'N/A'
        
        specratings = stuff = soup.find('div', {'class' : 'l-secondary-ratings-container l-clear l-margin-bottom-quarter l-m-margin-bottom-none'})
        try:
            hotel['cleanliness_rating'] = specratings.find_all('div')[2].text.strip()[:4]
        except:
            hotel['cleanliness_rating'] = 'N/A'
        try:
            hotel['service_rating'] = specratings.find_all('div')[7].text.strip()[:4]
        except:
            hotel['service_rating'] = 'N/A'
            
        hotel_list.append(hotel)
        print(hotel)
        print()
        print(datetime.now())
        print('='*80)
        print('='*80)
        print()
#         driver.close()
#         driver.switch_to.window(original_window)
#         time.sleep(1)

        counter += 1
        print(f'Collected {counter} hotels.')
    except:
        hotel = {}
        hotel['name'] = name
        hotel['url'] = url
        hotel_list.append(hotel)
        print(f"Number {x} didn't work at {datetime.now()}")
        continue
hotel_df = pd.DataFrame(hotel_list)
hotel_df    

{'brand': 'Luxury Collection', 'name': 'Sheraton Addis, a Luxury Collection Hotel, Addis Ababa', 'url': 'https://www.marriott.com/ADDLC', 'street': 'Taitu Street, P.O. Box 6002', 'locality': 'Addis Ababa', 'state': '', 'postalcode': '', 'coordinate1': 'Reviews', 'coordinate2': ' View Rates ', 'coordinate3': '', 'coordinate4': ' View Rates ', 'description': '[ Located in the heart of the Ethiopian capital, Sheraton Addis, a Luxury Collection Hotel, Addis Ababa sits on a hilltop overlooking the city, safely nestled between the National Palace and the prime minister’s residence. Our 5-star hotel in Ethiopia offers stunning views, acres of open space and a location just 20 minutes from Bole International Airport. Mixing African refinement with modern accents, the unique design of Sheraton Addis stands as the epitome of gracious living. Boasting glittering pools with enchanting underwater music, an indulgent spa menu of soothing massages at Aqva Club and several gourmet restaurants where ou

{'brand': 'Luxury Collection', 'name': 'The Grand Mansion, a Luxury Collection Hotel, Nanjing', 'url': 'https://www.marriott.com/NKGLC', 'street': 'No. 300 Changjiang Road', 'locality': 'Nanjing', 'state': '', 'postalcode': '210005', 'coordinate1': '32.042666', 'coordinate2': '118.798675', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ Situated in the storied cultural district of Nanjing, known as the Capital of Six Dynasties, our destination hotel captures the city's ancient charm and its modern sophistication, just steps from the Oriental Metropolitan Museum and Presidential Palace. Our state-of-the-art pool and Fitness Center cater to worldly guests. At the same time, our signature restaurants Xuan Ling Ge and The Atrium deliver rare dining experiences, while The Traveler Library and Tea Lounge present indigenous traditions. Beautifully appointed and spacious, the hotel's 158 guest rooms, including 15 suites, offer discerning amenities, modern technology and captivati

{'brand': 'Luxury Collection', 'name': 'ITC Grand Chola, a Luxury Collection Hotel, Chennai', 'url': 'https://www.marriott.com/MAALC', 'street': '#63 Mount Road, Guindy', 'locality': 'Chennai', 'state': '', 'postalcode': '600032', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/maalc-itc-grand-chola-a-luxury-collection-hotel-chennai/?directPageRequest=true', 'coordinate3': '13.010574', 'coordinate4': '80.220192', 'description': "[ This ornate, palatial tribute to Southern India's greatest empires – The Imperial Cholas – overlooks verdant foliage in the heart of Chennai. It embodies the highest standards in Indian hospitality balanced with elegant restraint in a prime property with distinct personality. ITC Grand Chola has 522 rooms along with 78 luxuriously appointed service apartments- collectively its 600 spacious guest rooms, suites and luxury service apartments are the epitome of Indian grace and style, expertly delegated with thoughtful amenit

{'brand': 'Luxury Collection', 'name': 'ITC Sonar, a Luxury Collection Hotel, Kolkata', 'url': 'https://www.marriott.com/CCULC', 'street': 'JBS Haldane Avenue (Opp Science City)', 'locality': 'Kolkata', 'state': '', 'postalcode': '700046', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/cculc-itc-sonar-a-luxury-collection-hotel-kolkata/?directPageRequest=true', 'coordinate3': '22.544253', 'coordinate4': '88.398109', 'description': '[ A verdant oasis just a short drive from Kolkata, our captivating setting inspires the mind while a spa, and more rejuvenate the body. ]', 'overall_rating': 'N/A', 'cleanliness_rating': '4.2', 'service_rating': '4.0'}

2020-09-21 22:55:57.722123

Collected 19 hotels.
{'brand': 'Luxury Collection', 'name': 'ITC Grand Central, a Luxury Collection Hotel, Mumbai', 'url': 'https://www.marriott.com/BOMRL', 'street': 'Dr Babasaheb Ambedkar Road, Parel', 'locality': 'Mumbai', 'state': '', 'postalcode': '400012', 'coordinate1': 

{'brand': 'Luxury Collection', 'name': 'The Laguna, a Luxury Collection Resort & Spa, Nusa Dua, Bali', 'url': 'https://www.marriott.com/DPSLC', 'street': 'Kawasan Pariwisata Nusa Dua Lot N2', 'locality': 'Nusa Dua', 'state': '', 'postalcode': '80363', 'coordinate1': 'Reviews', 'coordinate2': ' View Rates ', 'coordinate3': '', 'coordinate4': ' View Rates ', 'description': "[ From its panoramic views of the Indian Ocean to its Balinese hospitality, The Laguna, a Luxury Collection Resort and Spa, Nusa Dua, Bali offers a memorable experience. Our Nusa Dua beach resort provides spacious rooms, lagoon access suites and luxury villas. Amenities include stunning Balinese decor, indulgent signature beds, marble bathrooms, 24-hour butler service and extraordinary views of Bali. Pamper yourself in our tranquil resort spa, stay fit in our modern gym and take a dip in one of seven lagoon pools. A special children's pool is also available. Savor every bite and sip at our award-winning restaurants or

...

{'brand': 'Luxury Collection', 'name': 'Palace Hotel, a Luxury Collection Hotel, San Francisco', 'url': 'https://www.marriott.com/SFOLC', 'street': '2 New Montgomery Street', 'locality': 'San Francisco', 'state': 'California', 'postalcode': '94105', 'coordinate1': 'Reviews', 'coordinate2': ' View Rates ', 'coordinate3': '', 'coordinate4': ' View Rates ', 'description': '[ Encounter the luxury of the Gilded Age, historic architecture and world-class service at Palace Hotel, a Luxury Collection Hotel, San Francisco. Located in downtown San Francisco, our iconic, historic hotel features authentic elements paired with classically inspired, modern décor. Relish our wealth of hotel services and amenities, from professional concierge assistance and valet parking, to in-room dining, scenic city views, a cutting-edge fitness center, meetings and event services and more. Our legendary hotel showcases two famous restaurants, with a distinctive setting for any occasion. Enjoy a leisurely dip in ou

{'brand': 'Luxury Collection', 'name': 'The Liberty, a Luxury Collection Hotel, Boston', 'url': 'https://www.marriott.com/BOSLC', 'street': '215 Charles Street', 'locality': 'Boston', 'state': 'Massachusetts', 'postalcode': '02114', 'coordinate1': 'Reviews', 'coordinate2': ' View Rates ', 'coordinate3': '', 'coordinate4': ' View Rates ', 'description': '[ Embrace the extraordinary at The Liberty, a Luxury Collection Hotel, Boston. Once the storied Charles Street Jail, this hotel boasts gorgeous historic architecture and a central location at the foot of Beacon Hill, just steps from premier shopping boutiques and dining options. Enjoy convenient access to top attractions, such as Fenway Park, Freedom Trail, Boston Public Gardens and The Wang Center, as well as access to the Charles River Esplanade, ideal for running, biking, sailing and kayaking. Retreat to boutique rooms and suites bathed in natural light via floor-to-ceiling windows and offering stunning views of the majestic Boston s

{'brand': 'Luxury Collection', 'name': 'The Joseph, a Luxury Collection Hotel, Nashville', 'url': 'https://www.marriott.com/BNALJ', 'street': '401 Korean Veterans Boulevard', 'locality': 'Nashville', 'state': 'Tennessee', 'postalcode': '37203', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ The Joseph, a Luxury Collection Hotel, Nashville is a hotel of Southern origin and Southern heart. It brings the full story of Nashville to life, defined by the unique combination of cultural and artistic diversity, Southern hospitality and meaningful experiences. Authentic and comfortable, yet impeccably designed, our downtown hotel’s unique spaces invite the discerning traveler into a world of warm sophistication that delights and surprises while remaining decidedly Tennessean. From the arrival experience in the porte-cochere, to the handmade cocktails crafted on the 21st-floor bar, to the view of the skyline from the fitness center, to th

{'brand': 'Luxury Collection', 'name': 'Hotel Paracas, a Luxury Collection Resort, Paracas', 'url': 'https://www.marriott.com/PIOLC', 'street': 'Av. Paracas S/N', 'locality': 'Paracas', 'state': '', 'postalcode': '', 'coordinate1': ' View Rates ', 'coordinate2': '', 'coordinate3': ' View Rates ', 'coordinate4': 'https://www.marriott.com/hotels/maps/travel/piolc-hotel-paracas-a-luxury-collection-resort-paracas/?directPageRequest=true', 'description': '[ Located 3 hours south from Lima, facing the Paracas National Reserve, where natural beaches and a great variety of wildlife are preserved. The hotel offers world-class facilities, a private dock, the city’s only luxury spa, exquisite local and international cuisine and exciting experiences that will allow you to be in touch with nature. Its 120 rooms and suites are prepared to ensure the greatest comfort and the best rest. Each of them has a furnished terrace and a view of the sea or of the wonderful gardens. Paracas features plenty of a

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,Luxury Collection,"Sheraton Addis, a Luxury Collection Hotel, Add...",https://www.marriott.com/ADDLC,"Taitu Street, P.O. Box 6002",Addis Ababa,,,Reviews,View Rates,,View Rates,[ Located in the heart of the Ethiopian capita...,N/A,N/A,N/A
1,Luxury Collection,"North Island, a Luxury Collection Resort, Seyc...",https://www.marriott.com/SEZLC,No street address,North Island,,,N/A,N/A,N/A,N/A,[ Experience your own private island in the Se...,N/A,N/A,N/A
2,Luxury Collection,"The Alexander, a Luxury Collection Hotel, Yerevan",https://www.marriott.com/EVNLC,3/4 Abovyan Street,Yerevan,,0001,Reviews,https://www.marriott.com/hotels/maps/travel/ev...,40.180687,44.514998,[ Introducing an exclusive new address to the ...,N/A,5.0,5.0
3,Luxury Collection,"Meixi Lake Hotel, a Luxury Collection Hotel, C...",https://www.marriott.com/CSXLC,"1177 Huanhu Road, Meixi Lake, Yuelu District",Changsha,,410006,Reviews,https://www.marriott.com/hotels/maps/travel/cs...,28.194617,112.900904,[ Situated in the heart of the ancient city of...,N/A,4.6,4.5
4,Luxury Collection,"The Castle Hotel, a Luxury Collection Hotel, D...",https://www.marriott.com/DLCLC,"No. 600 Binhai West Road, Shahekou District",Dalian,,116023,38.88019988,121.5951325,N/A,N/A,[ Overlooking Xinghai Bay and the breathtaking...,N/A,4.6,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,Luxury Collection,"The Equinox, a Luxury Collection Golf Resort &...",https://www.marriott.com/RUTLC,3567 Main Street,Manchester Village,Vermont,05254,https://www.marriott.com/hotels/maps/travel/ru...,43.161991,-73.072563,N/A,[ A beacon of sophistication for the last 250 ...,N/A,4.3,4.3
121,Luxury Collection,"Park Tower, a Luxury Collection Hotel, Buenos ...",https://www.marriott.com/BUEPL,Avenida Leandro N. Alem 1193,Buenos Aires,,1001,Reviews,View Rates,,View Rates,"[ Culture, art and history collide in downtown...",N/A,N/A,N/A
122,Luxury Collection,"Palacio del Inka, a Luxury Collection Hotel, C...",https://www.marriott.com/CUZLC,Plazoleta Santo Domingo 259,Cusco,,,View Rates,,View Rates,https://www.marriott.com/hotels/maps/travel/cu...,"[ Considered a historic landmark, Palacio del ...",N/A,N/A,N/A
123,Luxury Collection,"Hotel Paracas, a Luxury Collection Resort, Par...",https://www.marriott.com/PIOLC,Av. Paracas S/N,Paracas,,,View Rates,,View Rates,https://www.marriott.com/hotels/maps/travel/pi...,"[ Located 3 hours south from Lima, facing the ...",N/A,N/A,N/A


In [6]:
hotel_df = hotel_df.drop_duplicates(subset='name')

In [7]:
hotel_df

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,Luxury Collection,"Sheraton Addis, a Luxury Collection Hotel, Add...",https://www.marriott.com/ADDLC,"Taitu Street, P.O. Box 6002",Addis Ababa,,,Reviews,View Rates,,View Rates,[ Located in the heart of the Ethiopian capita...,N/A,N/A,N/A
1,Luxury Collection,"North Island, a Luxury Collection Resort, Seyc...",https://www.marriott.com/SEZLC,No street address,North Island,,,N/A,N/A,N/A,N/A,[ Experience your own private island in the Se...,N/A,N/A,N/A
2,Luxury Collection,"The Alexander, a Luxury Collection Hotel, Yerevan",https://www.marriott.com/EVNLC,3/4 Abovyan Street,Yerevan,,0001,Reviews,https://www.marriott.com/hotels/maps/travel/ev...,40.180687,44.514998,[ Introducing an exclusive new address to the ...,N/A,5.0,5.0
3,Luxury Collection,"Meixi Lake Hotel, a Luxury Collection Hotel, C...",https://www.marriott.com/CSXLC,"1177 Huanhu Road, Meixi Lake, Yuelu District",Changsha,,410006,Reviews,https://www.marriott.com/hotels/maps/travel/cs...,28.194617,112.900904,[ Situated in the heart of the ancient city of...,N/A,4.6,4.5
4,Luxury Collection,"The Castle Hotel, a Luxury Collection Hotel, D...",https://www.marriott.com/DLCLC,"No. 600 Binhai West Road, Shahekou District",Dalian,,116023,38.88019988,121.5951325,N/A,N/A,[ Overlooking Xinghai Bay and the breathtaking...,N/A,4.6,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,Luxury Collection,"The Equinox, a Luxury Collection Golf Resort &...",https://www.marriott.com/RUTLC,3567 Main Street,Manchester Village,Vermont,05254,https://www.marriott.com/hotels/maps/travel/ru...,43.161991,-73.072563,N/A,[ A beacon of sophistication for the last 250 ...,N/A,4.3,4.3
121,Luxury Collection,"Park Tower, a Luxury Collection Hotel, Buenos ...",https://www.marriott.com/BUEPL,Avenida Leandro N. Alem 1193,Buenos Aires,,1001,Reviews,View Rates,,View Rates,"[ Culture, art and history collide in downtown...",N/A,N/A,N/A
122,Luxury Collection,"Palacio del Inka, a Luxury Collection Hotel, C...",https://www.marriott.com/CUZLC,Plazoleta Santo Domingo 259,Cusco,,,View Rates,,View Rates,https://www.marriott.com/hotels/maps/travel/cu...,"[ Considered a historic landmark, Palacio del ...",N/A,N/A,N/A
123,Luxury Collection,"Hotel Paracas, a Luxury Collection Resort, Par...",https://www.marriott.com/PIOLC,Av. Paracas S/N,Paracas,,,View Rates,,View Rates,https://www.marriott.com/hotels/maps/travel/pi...,"[ Located 3 hours south from Lima, facing the ...",N/A,N/A,N/A


In [8]:
hotel_df.to_csv('luxurycollection_description.csv')